<a href="https://colab.research.google.com/github/bvkakadiya/NoteProjectByBK/blob/master/ml_train_model_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 68kB/s 
     |████████████████████████████████| 3.2MB 54.4MB/s 
     |████████████████████████████████| 368kB 65.0MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [1]:
# To run and train model use below command 
# python3 train_model_ml.py  '-m' 'sl' '-e' '10' '-b' '256' '-alpha' '0.1' '-beta' '1.0' '-c' 'C5'

from __future__ import absolute_import
from __future__ import print_function
import os
import time
import numpy as np
import pandas as pd
import tensorflow.keras.backend as K
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.regularizers import l2
import argparse
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from loss import symmetric_cross_entropy, cross_entropy
from callback_util import LoggerCallback, SGDLearningRateTracker
from util import get_lr_scheduler
import distutils


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


In [0]:
Train_Data_Sample_Path = r'A_train_samples.xlsx'
X_data_loaded = pd.read_excel(Train_Data_Sample_Path)

In [0]:
Test_Data_label_Path = r'A_train_labels.xlsx'
Y_data_loaded =pd.read_excel(Test_Data_label_Path)

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input,Reshape, Conv1D, Dense, MaxPooling1D, Dropout, Flatten, Activation, BatchNormalization

def get_model(seq_len=None, input_features=None, num_classes=10):
    
    inputs = Input(shape=(input_features, 1),dtype=tf.float32, name='input_layer')   
    # x = inputs
    # x = Reshape((input_features, 1))(inputs)
    # Block 1
    # inputs = Input(name='input', shape=(2000, 42))
    x = Conv1D(64, 2, padding='same', kernel_initializer="he_normal", name='block1_conv1')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(128, 2, padding='same', kernel_initializer="he_normal", name='block1_conv2')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = MaxPooling1D(2, strides=2, name='block1_pool')(x)

    x = Flatten(name='flatten')(x)
 
    x = Dense(256, kernel_initializer="he_normal", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), name='fc1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu', name='lid')(x)

    x = Dense(num_classes, kernel_initializer="he_normal")(x)
    x = Activation(tf.nn.softmax)(x)

    # Create model.
    model = Model(inputs, x)

    return model



In [0]:

def train_input_fn(batch_size=1024):
    # convert the inputs to a Dataset.
    
    dataset = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32),y_train.astype(np.float32)))
    # tf.dtypes.cast(dataset, tf.float32)
    # shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

    # return the dataset.
    return dataset

def batch_generator(X_train,x_test,batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, 42)
        x_batch = np.zeros(shape=x_shape, dtype=np.float64)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, 1)
        y_batch = np.zeros(shape=y_shape, dtype=np.float64)
        num_train = len(X_train)
        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = X_train[idx:idx+sequence_length]
            y_batch[i] = x_test[idx:idx+sequence_length]

            yield (x_batch, y_batch)


In [0]:
sequence_length = 2000
    # load data
x =X_data_loaded.values
y = pd.DataFrame(Y_data_loaded['B']).values
X_train = MinMaxScaler().fit_transform(x)

# x, w, y, z = train_test_split(x,y,test_size = 0.40,train_size =0.60)
# X_train, x_validation, y_train, y_validation = train_test_split(x,y,test_size = 0.25,train_size =0.75)
y_train_clean = y
x_validation=x 
y_validation=y
y_train=y
seq_len = X_train.shape[0]
input_features = X_train.shape[1]
X_train = np.array(X_train).reshape(seq_len, input_features, 1)
num_classes = 10

In [0]:
# if distutils.version.LooseVersion(tf.__version__) < '1.14':
#     raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/shakespeare_with_tpu_and_keras.ipynb')

# TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# print(TPU_WORKER)

def train(model_name, batch_size, epochs, asym, alpha, beta, column_name):
    sequence_length = 2000
    # load data
    x =X_data_loaded.values
    y = pd.DataFrame(Y_data_loaded[column_name]).values
    X_train = MinMaxScaler().fit_transform(x)

    # x, w, y, z = train_test_split(x,y,test_size = 0.40,train_size =0.60)
    # X_train, x_validation, y_train, y_validation = train_test_split(x,y,test_size = 0.25,train_size =0.75)
    y_train_clean = y
    x_validation=x 
    y_validation=y
    y_train=y
    seq_len = X_train.shape[0]
    input_features = X_train.shape[1]
    num_classes = 10
   
    # X_train = np.array(X_train).reshape(seq_len, input_features, 1)
    # x_validation = np.array(x_validation).reshape(x_validation.shape[0], input_features, 1)
    generator = batch_generator(X_train,y_train, batch_size=batch_size,sequence_length=sequence_length)
    x_batch, y_batch = next(generator)
    print(x_batch.shape)
    print(y_batch.shape)
    print("seq_len", seq_len, "num_classes", num_classes, "input_features:", input_features)
    
    # load model
    # resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    # tf.contrib.distribute.initialize_tpu_system(resolver)
    # strategy = tf.contrib.distribute.TPUStrategy(resolver)
    # with strategy.scope():
    model = get_model(seq_len, input_features, num_classes=10)
    model.summary()
    
    # Select Optimizer
    optimizer = SGD(lr=0.1, decay=1e-4, momentum=0.9)

    # create loss
    loss = symmetric_cross_entropy(alpha,beta)
    
    # model
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['accuracy']
    )

    if asym:
        model_save_file = "asym_%s.{epoch:02d}.hdf5" % (model_name)
    else:
        model_save_file = "model_%s_{epoch:02d}_%s.hdf5" % (model_name, column_name)
    
    ## do real-time updates using callbakcs
    callbacks = []

    cp_callback = ModelCheckpoint(model_save_file,
                                    monitor='val_loss',
                                    verbose=0,
                                    save_best_only=False,
                                    save_weights_only=True,
                                    period=1)
    callbacks.append(cp_callback)
    
    # learning rate scheduler if use sgd
    lr_scheduler = get_lr_scheduler()
    callbacks.append(lr_scheduler)
    callbacks.append(SGDLearningRateTracker(model))
    
    # acc, loss, lid
    log_callback = LoggerCallback(model, X_train, y_train, y_train_clean, x_validation, y_validation, model_name, asym, epochs, alpha, beta)
    callbacks.append(log_callback)
    generator = batch_generator(X_train,y_train, batch_size=batch_size,sequence_length=sequence_length)
    # train model
    # model.fit(generator=generator,
    #             steps_per_epoch = int(len(X_train) / batch_size), 
    #             epochs=epochs, 
    #             validation_data = (x_validation, y_validation), 
    #             validation_steps = int(len(x_validation) / batch_size),
    #             verbose=1, 
    #             callbacks=callbacks)
    try:
        device_name = os.environ['COLAB_TPU_ADDR']
        TPU_ADDRESS = 'grpc://' + device_name
        print('Found TPU at: {}'.format(TPU_ADDRESS))
    except KeyError:
        print('TPU not found')
  
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(
        model,
        strategy=tf.contrib.tpu.TPUDistributionStrategy(
            tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS))
    )

    tpu_model.fit(train_input_fn,
                    epochs=10,
                    steps_per_epoch=int(len(X_train) / batch_size),
                    #validation_data=validation_data,
                    callbacks=callbacks)




In [16]:
train(model_name ='sl', batch_size =256, epochs =7, asym =False, alpha =0.1, beta =0.1, column_name = 'B')


(256, 2000, 42)
(256, 2000, 1)
seq_len 175341 num_classes 10 input_features: 42
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 42, 1)             0         
_________________________________________________________________
block1_conv1 (Conv1D)        (None, 42, 64)            192       
_________________________________________________________________
batch_normalization_v1_9 (Ba (None, 42, 64)            256       
_________________________________________________________________
activation_9 (Activation)    (None, 42, 64)            0         
_________________________________________________________________
block1_conv2 (Conv1D)        (None, 42, 128)           16512     
_________________________________________________________________
batch_normalization_v1_10 (B (None, 42, 128)           512       
______________________________________________________________